import library

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import NearestNeighbors
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
!pip install scikit-surprise
from surprise import KNNBasic
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

leoad dataset

In [ ]:
# Load dataset
books = pd.read_csv('Books.csv')
ratings = pd.read_csv('Ratings.csv')

<ipython-input-2-eb6bab3a1c5e>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv')


data preprocessing

In [ ]:
#menggabungkan data buku dan data ratings sesuai nilai ISBN
books_data=books.merge(ratings,on="ISBN")
df=books_data.copy()
#menghapus data kosong
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
#menghapus kolom yang tidak di gunakan
df.drop(columns=["Image-URL-S","Image-URL-M"],axis=1,inplace=True)
#menghapus rating yang bernilai 0
df.drop(index=df[df["Book-Rating"]==0].index,inplace=True)
#membersihkan kata pada kolom book title
df["Book-Title"]=df["Book-Title"].apply(lambda x: re.sub("[\W_]+"," ",x).strip())
#menghapus pengguna yang memberikan rate kurang dari 5000 kali.(agar data tidak terlalu banyak)
new_df=df[df['User-ID'].map(df['User-ID'].value_counts()) > 200]
#membuat
users_pivot=new_df.pivot_table(index=["User-ID"],columns=["Book-Title"],values="Book-Rating")
users_pivot.fillna(0,inplace=True)
books_pivot=new_df.pivot_table(index=["Book-Title"],columns=["User-ID"],values="Book-Rating")
books_pivot.fillna(0,inplace=True)

users_pivot


# Save the DataFrame to a CSV file
data_merge = pd.DataFrame(df)
data_merge.to_csv('data_merge.csv', index=False)


rating_count=pd.DataFrame(df["Book-Title"].value_counts())
rare_books=rating_count[rating_count["Book-Title"]<=200].index
common_books=df[~df["Book-Title"].isin(rare_books)]

In [ ]:
#menampilkan matriks user-item
books_pivot

User-ID,4385,6251,6575,7346,11676,13552,16634,16795,23768,23872,...,257204,258185,261829,262998,264321,265115,265889,269566,270713,274061
Book-Title,,,,,,,,,,,,,,,,,,,,,
01 01 00 A Novel of the Millennium,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01 01 00 The Novel of the Millennium,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1 000 Makers of the Millennium The Men and Women Who Have Shaped the Last 1 000 Years,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1 001 Old Time Garden Tips Timeless Bits of Wisdom on How to Grow Everything Organically from the Good Old Days When Everyone Did,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1 003 Great Things About Kids,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wet sand raven tracks,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
whataboutrick com a poetic tribute to Richard A Ricci,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Â Corre perro corre,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#tampilan dataset books
books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [ ]:
#tampilan dari dataset ratings
ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [ ]:
#tampilan final dataset
data_merge

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,User-ID,Book-Rating
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,8,5
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,11676,8
5,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,67544,8
8,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,116866,9
9,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,123629,9
...,...,...,...,...,...,...,...,...
1031116,0395264707,Dreamsnake,Vonda N. McIntyre,1978,Houghton Mifflin,http://images.amazon.com/images/P/0395264707.0...,275318,10
1031121,1845170423,Cocktail Classics,David Biggs,2004,Connaught,http://images.amazon.com/images/P/1845170423.0...,275970,7
1031123,0449906736,Flashpoints Promise and Peril in a New World,Robin Wright,1993,Ballantine Books,http://images.amazon.com/images/P/0449906736.0...,276313,5
1031124,0440400988,There s a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,276463,7


membuat function untuk prepare data dan popular books

In [ ]:
common_books = common_books.drop_duplicates(subset=["Book-Title"])
common_books.reset_index(inplace=True)
common_books["index"] = [i for i in range(common_books.shape[0])]
targets = ["Book-Title", "Book-Author", "Publisher"]
common_books["all_features"] = [" ".join(common_books[targets].iloc[i,].values) for i in range(common_books[targets].shape[0])]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(common_books["all_features"])

# Calculate cosine similarity
similarity = cosine_similarity(tfidf_matrix)



<ipython-input-8-3f236170ee45>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  common_books["index"] = [i for i in range(common_books.shape[0])]


In [ ]:
common_books["all_features"]

0                       The Testament John Grisham Dell
1                      Wild Animus Rich Shapero Too Far
2            Timeline MICHAEL CRICHTON Ballantine Books
3     To Kill a Mockingbird Harper Lee Little Brown ...
4                   The Street Lawyer JOHN GRISHAM Dell
5        The Joy Luck Club Amy Tan Prentice Hall (K-12)
6     The Catcher in the Rye J.D. Salinger Little, B...
7     The Five People You Meet in Heaven Mitch Albom...
8               Angels amp Demons Dan Brown Pocket Star
9     Bridget Jones s Diary Helen Fielding Picador (UK)
10          Fahrenheit 451 Ray Bradbury Distribooks Inc
11                  The Brethren JOHN GRISHAM Doubleday
12    Girl with a Pearl Earring Tracy Chevalier Plum...
13                      Life of Pi Yann Martel Harcourt
14    A Painted House John Grisham Dell Publishing C...
15    The Secret Life of Bees Sue Monk Kidd Penguin ...
16                The Firm John Grisham Doubleday Books
17    Divine Secrets of the Ya Ya Sisterhood A N

In [ ]:
# Create DataFrames
tf_table = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
idf_table = pd.DataFrame({'Term': tfidf_vectorizer.get_feature_names_out(), 'IDF': tfidf_vectorizer.idf_})
df_table = pd.DataFrame({'Term': tfidf_vectorizer.get_feature_names_out(), 'DF': (tfidf_matrix > 0).sum(axis=0).tolist()[0]})
tfidf_table = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_table = tfidf_table.mul(tfidf_vectorizer.idf_, axis=1)

# Display 5 rows of each table
print("TF Table:")
print(tf_table.head(5))

print("\nIDF Table:")
print(idf_table.head(5))

print("\nDF Table:")
print(df_table.head(5))

print("\nTF-IDF Table:")
print(tfidf_table.head(5))

TF Table:
    12  1st  451  albom  alice       amp  amy   an  and  andre  ...  weiner  \
0  0.0  0.0  0.0    0.0    0.0  0.000000  0.0  0.0  0.0    0.0  ...     0.0   
1  0.0  0.0  0.0    0.0    0.0  0.000000  0.0  0.0  0.0    0.0  ...     0.0   
2  0.0  0.0  0.0    0.0    0.0  0.000000  0.0  0.0  0.0    0.0  ...     0.0   
3  0.0  0.0  0.0    0.0    0.0  0.340321  0.0  0.0  0.0    0.0  ...     0.0   
4  0.0  0.0  0.0    0.0    0.0  0.000000  0.0  0.0  0.0    0.0  ...     0.0   

   wells  where  wife      wild  with   ya  yann  you  young  
0    0.0    0.0   0.0  0.000000   0.0  0.0   0.0  0.0    0.0  
1    0.0    0.0   0.0  0.408248   0.0  0.0   0.0  0.0    0.0  
2    0.0    0.0   0.0  0.000000   0.0  0.0   0.0  0.0    0.0  
3    0.0    0.0   0.0  0.000000   0.0  0.0   0.0  0.0    0.0  
4    0.0    0.0   0.0  0.000000   0.0  0.0   0.0  0.0    0.0  

[5 rows x 235 columns]

IDF Table:
    Term       IDF
0     12  4.258097
1    1st  4.258097
2    451  4.258097
3  albom  3.852631
4  ali

In [ ]:
def prepare_data():
    # Load dataset
    print("Books Shape: " ,books.shape )
    print("Ratings Shape: " ,ratings.shape )

    # Display the first few rows of the merged dataframe
    print("Gabungkan dataset book dan rating")
    print("dataset book")
    print(books.head(50))
    print("dataset rating")
    print(ratings.head(50))
    print("hasil penggabungan dataset")
    print(df.head(50))

    return books_data

def popular_books(df,n=100):
    rating_count=df.groupby("Book-Title").count()["Book-Rating"].reset_index()
    rating_count.rename(columns={"Book-Rating":"NumberOfVotes"},inplace=True)

    rating_average=df.groupby("Book-Title")["Book-Rating"].mean().reset_index()
    rating_average.rename(columns={"Book-Rating":"AverageRatings"},inplace=True)

    popularBooks=rating_count.merge(rating_average,on="Book-Title")

    def weighted_rate(x):
        v=x["NumberOfVotes"]
        R=x["AverageRatings"]

        return ((v*R) + (m*C)) / (v+m)

    C=popularBooks["AverageRatings"].mean()
    m=popularBooks["NumberOfVotes"].quantile(0.90)

    popularBooks=popularBooks[popularBooks["NumberOfVotes"] >=250]
    popularBooks["Popularity"]=popularBooks.apply(weighted_rate,axis=1)
    popularBooks=popularBooks.sort_values(by="Popularity",ascending=False)
    # Add the 'Image' column to the DataFrame
    image_urls = []
    for title in popularBooks["Book-Title"]:
        url = df.loc[df["Book-Title"] == title, "Image-URL-L"][:1].values[0]
        image_urls.append(url)

    popularBooks["Image"] = image_urls

    return popularBooks[["Book-Title", "NumberOfVotes", "AverageRatings", "Popularity", "Image"]].reset_index(drop=True).head(n)


membuat fucntion untuk metode colaborative filtering

In [ ]:

def users_choice(id):
    print("Provided ID:", id)

    # Check unique values in the "User-ID" column
    print("Unique User IDs:", new_df["User-ID"].unique())

    # Check entries with the provided ID
    user_entries = new_df[new_df["User-ID"].astype(str) == str(id)]
    print("User entries:", user_entries)

    # Check sorting step
    users_fav = user_entries.sort_values(["Book-Rating"], ascending=False)[:5]
    print("User favorites:", users_fav)
    return users_fav


def user_based(new_df,id):
    user_rec=[]
    similarities = []
    k=5
    if id not in new_df["User-ID"].values:
        print("❌ User NOT FOUND ❌")

    else:
        # Load the data into Surprise Dataset
        reader = Reader(rating_scale=(1, 10))
        data = Dataset.load_from_df(new_df[['User-ID', 'ISBN', 'Book-Rating']], reader)

        # Split the data into train and test sets
        trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

        # Build collaborative filtering model
        sim_options = {'name': 'cosine', 'user_based': True}
        model = KNNBasic(k=k, sim_options=sim_options)

        # Train the model on the training set
        model.fit(trainset)

        # Find the index of the target user in the matrix
        user_index = new_df[new_df['User-ID'] == id].index[0]

        # Find k-nearest neighbors and similarities
        user_neighbors = model.get_neighbors(user_index, k=5)
        similarities = [model.sim[user_index, i] for i in user_neighbors]
        print("simillarities",similarities)
        # Exclude the target user from the nearest neighbors
        neighbor_indices = user_neighbors[1:]
        for i in neighbor_indices:
            data = new_df[new_df["User-ID"] == new_df.iloc[i]['User-ID']]
            user_rec.extend(list(data.drop_duplicates("User-ID")["User-ID"].values))

    return user_rec,similarities

def common(new_df,user,user_id):
    x=new_df[new_df["User-ID"]==user_id]
    recommend_books=[]
    user=list(user)
    for i in user:
        y=new_df[(new_df["User-ID"]==i)]
        books=y.loc[~y["Book-Title"].isin(x["Book-Title"]),:]
        books=books.sort_values(["Book-Rating"],ascending=False)[0:5]
        recommend_books.extend(books["Book-Title"].values)

    return recommend_books[0:5]

# Collaborative Filtering
def collaborative_filtering(user_id):
    int(user_id)

    user_based_rec,similarities = user_based(new_df, user_id)
    books_for_user = common(new_df, user_based_rec, user_id)
    books_for_user_df = pd.DataFrame(books_for_user, columns=["Book-Title"])

    recommendations_list = []
    for i in range(5):
        book_title = books_for_user_df["Book-Title"].tolist()[i]
        rating_mean = round(df[df['Book-Title'] == book_title]['Book-Rating'].mean(), 1)
        img_url = new_df.loc[new_df["Book-Title"] == book_title, "Image-URL-L"][:1].values[0]
        similarity = similarities[i]
        recommendation = {
            "Book-Title": book_title,
            "Ratings": rating_mean,
            "Similarity": similarity,
            "Image": img_url
        }

        recommendations_list.append(recommendation)
    recommendations = pd.DataFrame(recommendations_list)

    recommendations["Image"] = recommendations["Image"].apply(lambda x: f'<img src="{x}" width="100">')

    return recommendations

membuat function untuk metode content based filtering

In [ ]:
# Content-Based Filtering
def content_based_filtering(bookTitle):
    bookTitle = str(bookTitle)
    if bookTitle in df["Book-Title"].values:
        rating_count = pd.DataFrame(df["Book-Title"].value_counts())
        rare_books = rating_count[rating_count["Book-Title"] <= 200].index
        common_books = df[~df["Book-Title"].isin(rare_books)]

        if bookTitle in rare_books:
            most_common = pd.Series(common_books["Book-Title"].unique()).sample(3).values
            print("No Recommendations for this Book ☹️ \n ")
            print("YOU MAY TRY: \n ")
            print("{}".format(most_common[0]), "\n")
            print("{}".format(most_common[1]), "\n")
            print("{}".format(most_common[2]), "\n")
        else:
            common_books = common_books.drop_duplicates(subset=["Book-Title"])
            common_books.reset_index(inplace=True)
            common_books["index"] = [i for i in range(common_books.shape[0])]
            targets = ["Book-Title", "Book-Author", "Publisher"]
            common_books["all_features"] = [" ".join(common_books[targets].iloc[i,].values) for i in range(common_books[targets].shape[0])]

                        # Calculate TF-IDF values
            tfidf_vectorizer = TfidfVectorizer()
            tfidf_matrix = tfidf_vectorizer.fit_transform(common_books["all_features"])

            # Calculate cosine similarity
            similarity = cosine_similarity(tfidf_matrix)
            index = common_books[common_books["Book-Title"] == bookTitle]["index"].values[0]
            similar_books = list(enumerate(similarity[index]))
            similar_books_sorted = sorted(similar_books, key=lambda x: x[1], reverse=True)[1:6]

            books = []
            ratings = []
            tfidf_values = []
            similarity_scores = []
            images = []

            for i in range(len(similar_books_sorted)):
                book_index = similar_books_sorted[i][0]
                book_data = common_books.loc[common_books["index"] == book_index]

                books.append(book_data["Book-Title"].item())
                ratings.append(df[df["Book-Title"] == books[i]]["Book-Rating"].mean())
                tfidf_values.append(tfidf_matrix[book_index, :].sum())
                similarity_scores.append(similar_books_sorted[i][1])
                images.append(book_data["Image-URL-L"].values[0])

            recommendations = pd.DataFrame({
                "Book-Title": books,
                "Ratings": ratings,
                "TF-IDF": tfidf_values,
                "Similarity": similarity_scores,
                "Image": images
            })
              # Use HTML to embed images in the DataFrame
            recommendations["Image"] = recommendations["Image"].apply(lambda x: f'<img src="{x}" width="100">')

            return recommendations

    else:
        print("❌ COULD NOT FIND ❌")

membuat fucntion untuk model hybrid filtering

In [ ]:
# Hybrid Filtering
def hybrid_filtering(user_id, book_title):
    # Collaborative Filtering
    collaborative_recommendations = collaborative_filtering(user_id)

    # Content-Based Filtering
    content_based_recommendations = content_based_filtering(book_title)

    # Create a DataFrame to store the hybrid recommendations
    hybrid_recommendations = pd.DataFrame(columns=["Book-Title", "Image", "Method", "Similarity","TF-IDF"])

    # Add collaborative recommendations to the hybrid DataFrame
    hybrid_recommendations = hybrid_recommendations.append(collaborative_recommendations.iloc[:, :4])
    hybrid_recommendations["Method"] = "Collaborative Filtering"
    hybrid_recommendations.reset_index(drop=True, inplace=True)

    # Add content-based recommendations to the hybrid DataFrame
    hybrid_recommendations = hybrid_recommendations.append(content_based_recommendations.iloc[:, :5])
    hybrid_recommendations["Method"] = np.where(hybrid_recommendations["Method"].isna(), "Content-Based Filtering", hybrid_recommendations["Method"])
    hybrid_recommendations.reset_index(drop=True, inplace=True)

    return hybrid_recommendations.head(10)

percoban rekomendasi

In [ ]:
#percobaan metode colaborative filtering
collaborative_filtering(11676)

Computing the cosine similarity matrix...
Done computing similarity matrix.
simillarities [1.0, 1.0, 1.0, 1.0, 1.0]


,Book-Title,Ratings,Similarity,Image
0,I Shouldn t Have To Scream More Than Once,10.0,1.0,"<img src=""http://images.amazon.com/images/P/07..."
1,Something Under the Bed Is Drooling,9.0,1.0,"<img src=""http://images.amazon.com/images/P/08..."
2,The Calvin amp Hobbes Lazy Sunday Book,8.8,1.0,"<img src=""http://images.amazon.com/images/P/08..."
3,Scientific Progress Goes Boink A Calvin and Ho...,9.1,1.0,"<img src=""http://images.amazon.com/images/P/08..."
4,The Revenge Of The Baby Sat,9.4,1.0,"<img src=""http://images.amazon.com/images/P/08..."


In [ ]:
#percobaan metode colaborative filtering
content_based_filtering('The Testament')

,Book-Title,Ratings,TF-IDF,Similarity,Image
0,The Street Lawyer,7.543689,2.339661,0.499912,"<img src=""http://images.amazon.com/images/P/04..."
1,The Summons,7.275081,2.553559,0.466111,"<img src=""http://images.amazon.com/images/P/04..."
2,A Time to Kill,8.009346,2.411616,0.435803,"<img src=""http://images.amazon.com/images/P/04..."
3,A Painted House,7.398907,2.608635,0.391300,"<img src=""http://images.amazon.com/images/P/04..."
4,The Brethren,7.441315,2.137095,0.372899,"<img src=""http://images.amazon.com/images/P/03..."


In [ ]:
#percobaan metode hybrid filtering
hybrid_filtering(11676,'The Testament')

Computing the cosine similarity matrix...
Done computing similarity matrix.
simillarities [1.0, 1.0, 1.0, 1.0, 1.0]


<ipython-input-13-ddf3489ed7ae>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hybrid_recommendations = hybrid_recommendations.append(collaborative_recommendations.iloc[:, :4])
<ipython-input-13-ddf3489ed7ae>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hybrid_recommendations = hybrid_recommendations.append(content_based_recommendations.iloc[:, :5])


,Book-Title,Image,Method,Similarity,TF-IDF,Ratings
0,I Shouldn t Have To Scream More Than Once,"<img src=""http://images.amazon.com/images/P/07...",Collaborative Filtering,1.000000,NaN,10.000000
1,Something Under the Bed Is Drooling,"<img src=""http://images.amazon.com/images/P/08...",Collaborative Filtering,1.000000,NaN,9.000000
2,The Calvin amp Hobbes Lazy Sunday Book,"<img src=""http://images.amazon.com/images/P/08...",Collaborative Filtering,1.000000,NaN,8.800000
3,Scientific Progress Goes Boink A Calvin and Ho...,"<img src=""http://images.amazon.com/images/P/08...",Collaborative Filtering,1.000000,NaN,9.100000
4,The Revenge Of The Baby Sat,"<img src=""http://images.amazon.com/images/P/08...",Collaborative Filtering,1.000000,NaN,9.400000
5,The Street Lawyer,"<img src=""http://images.amazon.com/images/P/04...",Content-Based Filtering,0.499912,2.339661,7.543689
6,The Summons,"<img src=""http://images.amazon.com/images/P/04...",Content-Based Filtering,0.466111,2.553559,7.275081
7,A Time to Kill,"<img src=""http://images.amazon.com/images/P/04...",Content-Based Filtering,0.435803,2.411616,8.009346
8,A Painted House,"<img src=""http://images.amazon.com/images/P/04...",Content-Based Filtering,0.391300,2.608635,7.398907
9,The Brethren,"<img src=""http://images.amazon.com/images/P/03...",Content-Based Filtering,0.372899,2.137095,7.441315


evaluasi model

In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

reader = Reader(rating_scale=(1, 10))

k_values=[5,10,20,30,40,50]

dfdf=df[df['User-ID'].map(df['User-ID'].value_counts()) > 5000]
# Load the data into Surprise Dataset
data = Dataset.load_from_df(dfdf[['User-ID', 'ISBN', 'Book-Rating']], reader)

# Split the data into train and test sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Membuat list untuk menyimpan hasil MAE
results = []

# Iterate over different values of k
for k in k_values:
    # Build collaborative filtering model
    sim_options = {'name': 'cosine', 'user_based': False}
    model = KNNBasic(k=k, sim_options=sim_options)

    # Train the model on the training set
    model.fit(trainset)

    # Make predictions on the test set
    predictions = model.test(testset)

    # Calculate MAE
    mae = accuracy.mae(predictions)

    # Append results to the list
    results.append({'k': k, 'MAE': mae})

# Create a DataFrame from the results list
results_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
results_df.to_csv('mae_results.csv', index=False)

Computing the cosine similarity matrix...
Done computing similarity matrix.
MAE:  1.2772
Computing the cosine similarity matrix...
Done computing similarity matrix.
MAE:  1.2769
Computing the cosine similarity matrix...
Done computing similarity matrix.
MAE:  1.2781
Computing the cosine similarity matrix...
Done computing similarity matrix.
MAE:  1.2770
Computing the cosine similarity matrix...
Done computing similarity matrix.
MAE:  1.2771
Computing the cosine similarity matrix...
Done computing similarity matrix.
MAE:  1.2765


In [ ]:
test_size_result=[]
for size in range(1,6):
    t_size=size/10
    trainset, testset = train_test_split(data, test_size=t_size, random_state=50)
    # Build collaborative filtering model
    sim_options = {'name': 'cosine', 'user_based': False}
    model = KNNBasic(k=5, sim_options=sim_options)

    # Train the model on the training set
    model.fit(trainset)

    # Make predictions on the test set
    predictions = model.test(testset)

    # Calculate MAE
    mae = accuracy.mae(predictions)

    # Append results to the list
    test_size_result.append({'test_size': t_size, 'MAE': mae})

# Create a DataFrame from the results list
test_size_result_df = pd.DataFrame(test_size_result)

# Save the DataFrame to a CSV file
test_size_result_df.to_csv('test_size_result.csv', index=False)

Computing the cosine similarity matrix...
Done computing similarity matrix.
MAE:  1.3567
Computing the cosine similarity matrix...
Done computing similarity matrix.
MAE:  1.3229
Computing the cosine similarity matrix...
Done computing similarity matrix.
MAE:  1.3100
Computing the cosine similarity matrix...
Done computing similarity matrix.
MAE:  1.2975
Computing the cosine similarity matrix...
Done computing similarity matrix.
MAE:  1.3146


In [ ]:
sim_matrix = model.compute_similarities()
predict = model.n_x
sim_matrix

Computing the cosine similarity matrix...
Done computing similarity matrix.


array([[1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 1., 1., 1.],
       ...,
       [0., 0., 1., ..., 1., 1., 1.],
       [0., 0., 1., ..., 1., 1., 1.],
       [0., 0., 1., ..., 1., 1., 1.]])